In [4]:
import pandas as pd
#Load les données du fichier excel
file_name="BenchMarksVE.xls"

In [8]:
df = pd.read_excel("BenchMarksVE.xls")

In [16]:
class SimpleConstructiveHeuristic:
    def __init__(self, items, params):
        self.items = items
        self.params = params
        self.I = range(len(items['quantities']))
        
    def find_best_lot(self, available_items):
        """Trouve le meilleur lot possible avec les articles disponibles"""
        model = LpProblem("BestLot", LpMaximize)
        
        # Variables de décision
        u = LpVariable.dicts("u", (i for i in self.I), cat='Binary')
        
        # Fonction objectif: maximiser le profit net du lot
        model += lpSum((self.items['prices'][i] - self.params['c_d']) * u[i] 
                      for i in self.I if available_items[i] > 0)
        
        # Contraintes
        # Nombre minimum et maximum d'articles par lot
        model += lpSum(u[i] for i in self.I) >= self.params['e_min']
        model += lpSum(u[i] for i in self.I) <= self.params['e_max']
        
        # Indice commercial minimum
        model += lpSum(self.items['indices'][i] * u[i] for i in self.I) >= self.params['r_min']
        
        # Articles disponibles
        for i in self.I:
            if available_items[i] == 0:
                model += u[i] == 0
                
        # Résolution
        try:
            model.solve(PULP_CBC_CMD(msg=False))
            
            if LpStatus[model.status] == 'Optimal':
                return {i: value(u[i]) for i in self.I}
            return None
            
        except Exception as e:
            print(f"Erreur lors de la résolution du lot: {str(e)}")
            return None
        
    def solve(self):
        """Implémente l'heuristique constructive simple"""
        solution = []
        available_items = self.items['quantities'].copy()
        
        # Tant qu'on peut créer des lots et qu'on n'a pas atteint le nombre max de types
        while len(solution) < self.params['L_max']:
            # Trouver le meilleur lot possible
            lot = self.find_best_lot(available_items)
            
            # Si aucun lot valide n'est trouvé, arrêter
            if lot is None:
                break
                
            # Calculer combien de ces lots peuvent être faits
            min_quantity = float('inf')
            articles_in_lot = False
            
            for i in self.I:
                if lot[i] > 0:
                    articles_in_lot = True
                    min_quantity = min(min_quantity, available_items[i])
            
            # Si le lot est vide ou impossible à réaliser, arrêter
            if not articles_in_lot or min_quantity == float('inf'):
                break
                    
            # Ajouter le lot à la solution
            solution.append((lot, min_quantity))
            
            # Mettre à jour les quantités disponibles
            for i in self.I:
                if lot[i] > 0:
                    available_items[i] -= min_quantity
            
            # Vérifier s'il reste assez d'articles pour faire un nouveau lot
            remaining_articles = sum(1 for i in self.I if available_items[i] > 0)
            if remaining_articles < self.params['e_min']:
                break
                    
        return solution

    def calculate_profit(self, solution):
        """Calcule le profit total d'une solution"""
        total_profit = 0
        for lot, quantity in solution:
            lot_profit = 0
            for i in self.I:
                if lot[i] > 0:
                    price = self.items['prices'][i]
                    profit = (price - self.params['c_d']) * quantity
                    lot_profit += profit
            total_profit += lot_profit
        return total_profit

In [47]:
def read_benchmarks_excel(filename):
    """Lit et nettoie les données du fichier Excel BenchMarksVE.xls"""
    try:
        # Lecture des données articles
        data1 = pd.read_excel(filename, usecols="B:G")
        data1.rename(columns={
            'Unnamed: 1': 'Article',
            'Unnamed: 2': 'di',
            'Unnamed: 3': 'Prix_Vente',
            'Unnamed: 4': 'Indice',
            'Unnamed: 5': 'NOMBRE_CUMULEE',
            'Unnamed: 6': 'multi'
        }, inplace=True)
        
        # Nettoyage des données
        data1 = data1.drop(2)
        data1 = data1.dropna()
        data1 = data1.reset_index(drop=True)
        
        # Conversion des types
        data1[['Prix_Vente', 'multi']] = data1[['Prix_Vente', 'multi']].astype(float)
        data1[['Article', 'di', 'Indice', 'NOMBRE_CUMULEE']] = data1[['Article', 'di', 'Indice', 'NOMBRE_CUMULEE']].astype(int)
        
        # Lecture des paramètres
        data2 = pd.read_excel(filename, usecols="M:N")
        data2.rename(columns={
            'Unnamed: 12': 'Paramètre_nom',
            'Unnamed: 13': 'Paramètre_num'
        }, inplace=True)
        data2 = data2.dropna()
        data2 = data2.reset_index(drop=True)
        data2['Paramètre_num'] = data2['Paramètre_num'].astype(float)
        
        # Création du dictionnaire des paramètres
        params = {}
        for _, row in data2.iterrows():
            param_name = row['Paramètre_nom'].strip().replace(' =', '')
            params[param_name] = row['Paramètre_num']
        
        # Lecture des postes de conditionnement
        data3 = pd.read_excel(filename, usecols="P:Q")
        data3 = data3.dropna()
        data3.rename(columns={
            'Unnamed: 15': 'numéro_poste',
            'Unnamed: 16': 'capacité'
        }, inplace=True)
        data3['capacité'] = data3['capacité'].astype(int)
        
        # Création des dictionnaires de retour
        items = {
            'quantities': data1['di'].tolist(),
            'prices': data1['Prix_Vente'].tolist(),
            'indices': data1['Indice'].tolist()
        }
        
        params = {
            'e_min': int(params['emin']),
            'e_max': int(params['emax']),
            'L_max': int(params['Lmax']),
            'c_d': float(params['cd']),
            'r_min': int(params['rmin'])
        }
        
        # Dictionnaire des postes
        stations = {i: cap for i, cap in enumerate(data3['capacité'].tolist())}
        
        return items, params, stations
        
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier: {str(e)}")
        return None, None, None

# Utilisation:
if __name__ == "__main__":
    items, params, stations = read_benchmarks_excel('BenchMarksVE.xls')
    
    if items is not None:
        print("\nDonnées chargées avec succès:")
        print(f"Nombre d'articles: {len(items['quantities'])}")
        print(f"Paramètres: {params}")
        print(f"Stations: {stations}")
        
        # Création des instances des classes de résolution
        heuristic = SimpleConstructiveHeuristic(items, params)
        solution = heuristic.solve()
        
# Afficher les résultats de l'heuristique
    print("\nSolution heuristique :")
    total_profit = 0
    for idx, (lot, quantity) in enumerate(solution):
        print(f"\nLot type {idx+1} (quantité: {quantity}):")
        lot_profit = 0
        for i in lot:
            if lot[i] > 0:
                price = items['prices'][i]
                profit = (price - params['c_d']) * quantity
                lot_profit += profit
                print(f"Article {i+1}: {lot[i]} (profit: {profit:.2f}€)")
        total_profit += lot_profit
        print(f"Profit du lot: {lot_profit:.2f}€")
    
    print(f"\nProfit total de l'heuristique: {total_profit:.2f}€")
    
    # Calculer la borne supérieure
    #print("\nCalcul de la borne supérieure...")
    #upper_bound = PrePackingUpperBound(items, params)
    #bound = upper_bound.subgradient_method()
    #print(f"Borne supérieure: {bound:.2f}€")
    
    # Calculer le gap
    # gap = ((bound - total_profit) / bound) * 100 if bound > 0 else 0
    # print(f"Gap: {gap:.2f}%")


Données chargées avec succès:
Nombre d'articles: 25
Paramètres: {'e_min': 2, 'e_max': 4, 'L_max': 15, 'c_d': 0.13, 'r_min': 15}
Stations: {0: 50000, 1: 50000, 2: 60000, 3: 60000, 4: 70000}

Solution heuristique :

Lot type 1 (quantité: 6403):
Article 1: 1.0 (profit: 29389.77€)
Article 2: 1.0 (profit: 28365.29€)
Article 3: 1.0 (profit: 27917.08€)
Article 4: 1.0 (profit: 27020.66€)
Profit du lot: 112692.80€

Lot type 2 (quantité: 2277):
Article 1: 1.0 (profit: 10451.43€)
Article 2: 1.0 (profit: 10087.11€)
Article 4: 1.0 (profit: 9608.94€)
Article 5: 1.0 (profit: 8880.30€)
Profit du lot: 39027.78€

Lot type 3 (quantité: 4008):
Article 1: 1.0 (profit: 18396.72€)
Article 2: 1.0 (profit: 17755.44€)
Article 5: 1.0 (profit: 15631.20€)
Article 6: 1.0 (profit: 14829.60€)
Profit du lot: 66612.96€

Lot type 4 (quantité: 3222):
Article 2: 1.0 (profit: 14273.46€)
Article 5: 1.0 (profit: 12565.80€)
Article 6: 1.0 (profit: 11921.40€)
Article 7: 1.0 (profit: 11760.30€)
Profit du lot: 50520.96€

Lot ty